### Load In

In [1]:
''' Make sure you have pandorasim installed and updated '''

import pandorasat as ps
import pandorasim as pp
ps.utils.get_phoenix_model(teff=7000, jmag=10) #this is a temporary fix to an import error with pandorasim
from pandorasim import VisibleSim, NIRSim
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates import SkyCoord
from astropy.io import fits
from pandorasat.plotting import animate
from pandorasat.plotting import save_mp4, save_gif
from astropy.time import Time, TimeDelta

### Importing additional functions and constants used in this notebook
import sys, os
sys.path.append(os.path.abspath('..'))
from CommissFunctions import generate_task_plan
from CommissFunctions import data_rate, bits_per_pix_VIS, compression_fractor_VIS, frame_time_VIS, stored_frames_per_int_VIS, pass_time_min


/Users/lindseywiser/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
''' Define a PandoraSat object for calling constants '''
p = ps.PandoraSat()

# Data Volume

Estimate data volume based on obseravtion parameters and estimated rates and sizes.

In [ ]:
''' this sets observing parameters for activity '''
VIS_xpix              = p.VISDA.shape[0] ### We collect the full VIS FOV for this task
VIS_ypix              = p.VISDA.shape[1]
regions               = 1
frames_per_int        = 1
num_int               = 200

''' Calculating data volumes '''
int_and_reset_time    = frame_time_VIS * frames_per_int
bits_per_int          = VIS_xpix * VIS_ypix * regions * bits_per_pix_VIS * stored_frames_per_int_VIS
bits_per_sec          = bits_per_int / int_and_reset_time
test_time             = num_int * int_and_reset_time
bits_per_sec_comp     = bits_per_sec / compression_fractor_VIS
bits_test_total       = bits_per_sec_comp * test_time
Gbits_test_packet     = bits_test_total / 1E9 * 1.1 * 1.25
downlinks             = Gbits_test_packet * 1E9 /(data_rate * 1E6) /pass_time_min /60

print('test time:', test_time, 'seconds')
print(Gbits_test_packet, 'Gbits')
print(downlinks, 'downlinks')

# Define Targets

For now, we select an arbitrary primary target to place at the center of the frame. The target selection process is being developed (check with Aisha Iyer or Lindsey Wiser).  

In [ ]:
### Posible Targets:
# ...
# ...
# ...
# ...

In [ ]:
''' Finds target coordinates based on astropy '''
### You can do this differently as long as you have DEC and RA
target = "GJ 3470"
c = SkyCoord.from_name(target)


''' Identify the start time and end time of the observation '''
start_time = Time.now() 
time_delta = TimeDelta(test_time, format='sec')
end_time = start_time + time_delta


''' Initialize a simulator object '''
sim = VisibleSim(ROI_size=(VIS_xpix, VIS_ypix), nROIs=regions) 


''' Point the simulator at the above target '''
###        RA,   DEC,    Theta 
sim.point(c.ra, c.dec, 0*u.deg)

In [ ]:
''' sim has various attributes you can print out '''
#print(sim.ra, sim.dec, sim.wcs)

''' Notably we have a Catalog of objects visable in the VIS FOV '''
cat = sim.source_catalog
#print(cat)

# Observation

In [ ]:
''' Plot the Full Frame Image.'''
sim.show_FFI()

In [ ]:
''' Create an Observation. '''

### This returns an astropy.io.fits.HDUList array. 
# `nreads` sets how many reads of the detector are coadded together to create a frame.
# `bin_frames` is a shortcut parameter to speed up computation. Must be a factor of nreads. We can ignore this for low nreads. 
# `nframes` sets how many frames will be returned.
# `start_time` indicates the time of observation. 
# `output_type` is set to array, for now. This make it easy to call in the notebook, but we'll save it as a fits file later. 
###

data = sim.observe(nreads=frames_per_int, bin_frames=1, nframes=num_int, start_time=start_time, output_type="array")

### The shape is: (num ROIs, num frames, nrows, ncolumns)
data.shape

In [ ]:
'''Animate over the integrations'''
### This is working, but since it is a full frame image object are very small and faint.
#animate(data[0])

### You can save this
#save_mp4(data[0], "output.mp4")
#save_gif(data[0], "output.gif")

# Save the simulated observation

In [ ]:
''' Re-Create the observation in the correct format for the .fits file (i.e., output_type set to default) '''
### Before, object_type was array for easy use in this notebook, but that is not correct for the .fits file
hdulist = sim.observe(nreads=frames_per_int, nframes=num_int, bin_frames=1, start_time=start_time)


''' Save our simulated observation in a fits file '''
targ = target.replace(" ", "")
ffis_outfile = "3.10_ffis_"+str(targ)+".fits"   
hdulist.writeto(ffis_outfile, overwrite=True)

In [ ]:
''' Show what is in our fits file: 

FITS file extensions: 
    (1) PRIMARY with some information about the simulated observation 
    (2) SCIENCE with information about the data
    (3) ROIs with information about each ROI 
'''
### extension 3 is for ROIs. For this observation, we are taking a full frame, so we will not have ROIs. 

hdul_ffis = fits.open(ffis_outfile)
print(hdul_ffis.info())

#print('.')
#print('.')
#print('.')
#print('-- PRIMARY Headers --')
#print(hdul_ffis[0].header)

#print('.')
#print('.')
#print('.')
#print('-- SCIENCE Headers --')
#print(hdul_ffis[1].header)

#print('.')
#print('.')
#print('.')
#print('-- ROI Headers --')
#print(hdul_ffis[2].header)

# Data Products

### B2:  Generate a table that provides the mapping between the sky and detector positions to quantify field distortions

### B3:  Generate a table that provides the mapping between the sky and detector positions to quantify boresight alignment

Right now, we aren't including much here. We can develop this further if it would be useful. 

In [ ]:
''' Plot the FFIs from the fits file to compare to expected target locations '''

frame = 0 ### specify which integration to plot - can plot different integrations to see possible jitter
image = hdul_ffis[1].data[frame]

flipped = np.flipud(image) ### flipping the array to match the corrent pixel numbers on the detector
#plt.imshow(flipped, cmap='binary', interpolation='nearest', extent=[0, p.VISDA.shape[0], 0, p.VISDA.shape[1]])

#plt.xlabel('x pixel')
#plt.ylabel('y pixel')
#plt.title('Integration '+str(frame))

# Generate SOC File

In [ ]:
'''Here we define the information required for scheduling the observation.'''
### Some of this may change

variables = {
    'visit_id': '0310', ### setting based on task number, for now. Could change.
    'obs_id': '000', ### this adjusts for tasks with multiple observations
    'target': target, 
    'priority': 1, 
    'start_time': start_time, 
    'stop_time': end_time, 
    'RA': c.ra,
    'DEC': c.dec, 
    'NIR_AvgGroups': '', ### no NIR. 
    'NIR_ROI_StartX': '', 
    'NIR_ROI_StartY': '', 
    'NIR_ROI_SizeX': '', 
    'NIR_ROI_SizeY': '',
    'NIR_SC_Resets1': '',
    'NIR_SC_Resets2': '', 
    'NIR_SC_DropFrames1': '',
    'NIR_SC_DropFrames2': '',
    'NIR_SC_DropFrames3': '', 
    'NIR_SC_ReadFrames': '',
    'NIR_targetID': '', 
    'NIR_SC_Groups': '',
    'NIR_SC_Integrations': '', 
    'VIS_StarRoiDetMethod': 0,  ### 0: monitor stars at positions defined in [PredefinedStarRoiRa, PredefinedStarRoiDec]. 1: Run star detection algorithm on first image and obtain star ROIs from results.
    'VIS_FramesPerCoadd': frames_per_int, 
    'VIS_NumTotalFramesRequested': num_int, 
    'VIS_TargetRA': c.ra, 
    'VIS_TargetDEC': c.dec, 
    'VIS_IncludeFieldSolnsInResp': 1, ### Determines whether the Ra/Dec/Rot values calculated for each frame in the visible science collect will be included in the response message [1] or not [0].
    'VIS_StarRoiDimension': [VIS_xpix, VIS_ypix], 
    'VIS_MaxNumStarRois': '', ### max number of star ROis that will be used for coadding signal from target stars if StarRoiDetMethod == 1.
    'VIS_numPredefinedStarRois': 1, ### If StarRoiDetMethod==0, this is the number of [RA, DEC] coordinates contained in the PredefinedStarRoiRA and PredefinedStarRoiDEC vectors.
    'VIS_PredefinedStarRoiRa': c.ra, 
    'VIS_PredefinedStarRoiDec': c.dec, 
    'VIS_targetID': target, 
    'VIS_NumExposuresMax': num_int, 
    'VIS_ExposureTime_us': frame_time_VIS * 1000000  
}

In [ ]:
'''Generate the SOC .xml file'''
### This may also change format

output_soc_file = '3.10_'+variables['visit_id']+'_'+variables['obs_id']+'_SOC.xml' 
generate_task_plan(variables, output_soc_file)